In [3]:
import cv2
import numpy as np
import tensorflow as tf
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def build_generator():
    """ Define a simple CNN generator for watermark prediction """
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

generator = build_generator()
weights_path = "./model_weights/generator_epoch_4900.weights.h5"

if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

INPUT_VIDEO_PATH = "./input1.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked1.mp4"


cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
alpha = 0.01

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  
    frame = frame.astype(np.float32) / 255.0  
    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0  
    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]  
    watermark_resized = cv2.resize(watermark, (frame_width, frame_height))

    watermarked_frame = frame + alpha * np.expand_dims(watermark_resized, axis=-1)
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)

    out.write(watermarked_frame)

    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)
    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)
    frame_count += 1

cap.release()
out.release()

avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)


print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"📊  PSNR: {avg_psnr:.4f} (More Than 30 is better)")
print(f"📊  SSIM : {avg_ssim:.6f} (Closer to 1 is better)")



✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━